In [1]:
import time
import json
import joblib
from pprint import pprint

from pathlib import Path

import pandas as pd
import numpy as np

# import h5py
# from io import BytesIO
# from PIL import Image

# import torch
# from torch.utils.data import DataLoader, Dataset
# import torch.nn as nn
# import torch.nn.functional as F

# from timm import create_model

# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# from accelerate import Accelerator

from isic_helper import DotDict

In [2]:
# model_names = ["lgb", "cb", "xgb", "efficientnet_b0", "mobilevitv2_200"]
# versions = ["v5", "v2", "v1", "v3", "v1"]
# paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-train" for model_name, version in zip(model_names, versions)]

# weights = [10.0, 2.1134490467525704, 10.0, 6.841848422723057, 5.444051069089391]

model_names = ["xgb", "xgb"]
versions = ["v1", "v2"]
modes = ["train", "train"]
paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-{mode}" for model_name, version, mode in zip(model_names, versions, modes)]

weights = [6.73414629518269, 5.721006401618444]

SAMPLE_SIZE = 5000
EXPECTED_TEST_SIZE = 500000

In [3]:
id_column = "isic_id"
target_column = "target"
group_column = "patient_id"
fold_column = "fold"

numerical_features = [
    "age_approx",
    "clin_size_long_diam_mm",
    "tbp_lv_A", "tbp_lv_Aext",
    "tbp_lv_B", "tbp_lv_Bext",
    "tbp_lv_C", "tbp_lv_Cext",
    "tbp_lv_H", "tbp_lv_Hext",
    "tbp_lv_L", "tbp_lv_Lext",
    "tbp_lv_areaMM2",
    "tbp_lv_area_perim_ratio",
    "tbp_lv_color_std_mean",
    "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", "tbp_lv_deltaLB", "tbp_lv_deltaLBnorm",
    "tbp_lv_eccentricity",
    "tbp_lv_minorAxisMM",
    "tbp_lv_nevi_confidence",
    "tbp_lv_norm_border", "tbp_lv_norm_color",
    "tbp_lv_perimeterMM",
    "tbp_lv_radial_color_std_max",
    "tbp_lv_stdL", "tbp_lv_stdLExt",
    "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle",
    "tbp_lv_x", "tbp_lv_y", "tbp_lv_z",
]

ord_categorical_features = [
    "sex",
    "tbp_lv_location",
    "tbp_tile_type",
    "tbp_lv_location_simple",
]

ohe_categorical_features = [
    "anatom_site_general", 
    "attribution",
]

attribution_mapper = {
    "Memorial Sloan Kettering Cancer Center": "MSKCC",
    "ACEMID MIA": "ACEMIDMIA",
    "Department of Dermatology, Hospital Clínic de Barcelona": "DoD_HCB",
    "University Hospital of Basel": "UHB",
    "Frazer Institute, The University of Queensland, Dermatology Research Centre": "FI_TUQ-DRC",
    "Department of Dermatology, University of Athens, Andreas Syggros Hospital of Skin and Venereal Diseases, Alexander Stratigos, Konstantinos Liopyris": "DoD_UA",
    "ViDIR Group, Department of Dermatology, Medical University of Vienna": "ViDIR"
}

def preprocess(df):
    df["anatom_site_general"] = df["anatom_site_general"].fillna("missing_site")
    df["sex"] = df["sex"].fillna("missing_sex")
    df["tbp_tile_type"] = df["tbp_tile_type"].map({"3D: white": "white", "3D: XP": "XP"})
    df["attribution"] = df["attribution"].map(attribution_mapper)
    return df

def norm_feature(df, value_col, group_cols=[group_column], err=1e-5):
    stats = ["mean", "std"]
    tmp = df.groupby(group_cols)[value_col].agg(stats)
    tmp.columns = [f"{value_col}_{stat}" for stat in stats]
    tmp.reset_index(inplace=True)
    df = df.merge(tmp, on=group_cols, how="left")
    feature_name = f"{value_col}_patient_norm"
    df[feature_name] = ((df[value_col] - df[f"{value_col}_mean"]) / 
                                       (df[f"{value_col}_std"] + err))
    return df, feature_name

def count_features(df, col):
    tmp = df[[id_column, group_column, col]].pivot_table(
        values=id_column, 
        index=group_column, 
        columns=col, 
        aggfunc="count", 
        fill_value=0)
    feature_cols = tmp.columns.tolist()
    tmp.reset_index(inplace=True)
    tmp.index.name = None
    df = df.merge(tmp, on=group_column, how="left")
    return df, feature_cols

def feature_engineering(df, err=1e-5):
    new_num_cols = []
    
    df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    new_num_cols += ["lesion_size_ratio"]
    
    df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / df["tbp_lv_perimeterMM"]**2
    new_num_cols += ["lesion_shape_index"]
    
    df["hue_contrast"] = np.abs(df["tbp_lv_H"] - df["tbp_lv_Hext"])
    new_num_cols += ["hue_contrast"]
    
    df["luminance_contrast"] = np.abs(df["tbp_lv_L"] - df["tbp_lv_Lext"])
    new_num_cols += ["luminance_contrast"]
    
    df["lesion_color_difference"] = np.sqrt(df["tbp_lv_deltaA"]**2 +
                                            df["tbp_lv_deltaB"]**2 +
                                            df["tbp_lv_deltaL"]**2)
    new_num_cols += ["lesion_color_difference"]
    
    df["border_complexity"] = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    new_num_cols += ["border_complexity"]
    
    df["color_uniformity"] = df["tbp_lv_color_std_mean"] / (df["tbp_lv_radial_color_std_max"] + err)
    new_num_cols += ["color_uniformity"]
    
    df["position_distance_3d"] = np.sqrt(df["tbp_lv_x"]**2 +
                                         df["tbp_lv_y"]**2 +
                                         df["tbp_lv_z"]**2)
    new_num_cols += ["position_distance_3d"]
    
    df["perimeter_to_area_ratio"] = df["tbp_lv_perimeterMM"] / df["tbp_lv_areaMM2"]
    new_num_cols += ["perimeter_to_area_ratio"]
    
    df["area_to_perimeter_ratio"] = df["tbp_lv_areaMM2"] / df["tbp_lv_perimeterMM"]
    new_num_cols += ["area_to_perimeter_ratio"]
    
    df["lesion_visibility_score"] = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"]
    new_num_cols += ["lesion_visibility_score"]
    
    df["symmetry_border_consistency"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    new_num_cols += ["symmetry_border_consistency"]
    
    df["consistency_symmetry_border"] = (df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"] /
                                         (df["tbp_lv_symm_2axis"] + df["tbp_lv_norm_border"]))
    new_num_cols += ["consistency_symmetry_border"]
    
    df["color_consistency"] = df["tbp_lv_stdL"] / df["tbp_lv_Lext"]
    new_num_cols += ["color_consistency"]
    
    df["consistency_color"] = (df["tbp_lv_stdL"] * df["tbp_lv_Lext"] /
                               (df["tbp_lv_stdL"] * df["tbp_lv_Lext"]))
    new_num_cols += ["consistency_color"]
    
    df["size_age_interaction"] = df["clin_size_long_diam_mm"] * df["age_approx"]
    new_num_cols += ["size_age_interaction"]
    
    df["hue_color_std_interaction"] = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    new_num_cols += ["hue_color_std_interaction"]
    
    df["lesion_severity_index"] = (df["tbp_lv_norm_border"] +
                                   df["tbp_lv_norm_color"] +
                                   df["tbp_lv_eccentricity"]) / 3
    new_num_cols += ["lesion_severity_index"]
    
    df["shape_complexity_index"] = df["border_complexity"] + df["lesion_shape_index"]
    new_num_cols += ["shape_complexity_index"]
    
    df["color_contrast_index"] = (df["tbp_lv_deltaA"] +
                                  df["tbp_lv_deltaB"] + 
                                  df["tbp_lv_deltaL"] +
                                  df["tbp_lv_deltaLBnorm"])
    new_num_cols += ["color_contrast_index"]
    
    df["log_lesion_area"] = np.log1p(df["tbp_lv_areaMM2"])
    new_num_cols += ["log_lesion_area"]
    
    df["normalized_lesion_size"] = df["clin_size_long_diam_mm"] / df["age_approx"]
    new_num_cols += ["normalized_lesion_size"]
    
    df["mean_hue_difference"] = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    new_num_cols += ["mean_hue_difference"]
    
    df["std_dev_contrast"] = np.sqrt((df["tbp_lv_deltaA"]**2 +
                                      df["tbp_lv_deltaB"]**2 + 
                                      df["tbp_lv_deltaL"]**2) / 3)
    new_num_cols += ["std_dev_contrast"]
    
    df["color_shape_composite_index"] = (df["tbp_lv_color_std_mean"] + 
                                         df["tbp_lv_area_perim_ratio"] +
                                         df["tbp_lv_symm_2axis"]) / 3
    new_num_cols += ["color_shape_composite_index"]
    
    df["lesion_orientation_3d"] = np.arctan2(df["tbp_lv_y"], df["tbp_lv_x"])
    new_num_cols += ["lesion_orientation_3d"]
    
    df["overall_color_difference"] = (df["tbp_lv_deltaA"] + 
                                      df["tbp_lv_deltaB"] + 
                                      df["tbp_lv_deltaL"]) / 3
    new_num_cols += ["overall_color_difference"]
    
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    new_num_cols += ["symmetry_perimeter_interaction"]
    
    df["comprehensive_lesion_index"] = (df["tbp_lv_area_perim_ratio"] +
                                        df["tbp_lv_eccentricity"] +
                                        df["tbp_lv_norm_color"] +
                                        df["tbp_lv_symm_2axis"]) / 4
    new_num_cols += ["comprehensive_lesion_index"]
    
    df["color_variance_ratio"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_stdLExt"]
    new_num_cols += ["color_variance_ratio"]
    
    df["border_color_interaction"] = df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]
    new_num_cols += ["border_color_interaction"]
    
    df["border_color_interaction_2"] = ((df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]) /
                                        (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"]))
    new_num_cols += ["border_color_interaction_2"]
    
    df["size_color_contrast_ratio"] = df["clin_size_long_diam_mm"] / df["tbp_lv_deltaLBnorm"]
    new_num_cols += ["size_color_contrast_ratio"]
    
    df["age_normalized_nevi_confidence"] = df["tbp_lv_nevi_confidence"] / df["age_approx"]
    new_num_cols += ["age_normalized_nevi_confidence"]
    
    df["age_normalized_nevi_confidence_2"] = np.sqrt(df["tbp_lv_nevi_confidence"]**2 + df["age_approx"]**2)
    new_num_cols += ["age_normalized_nevi_confidence_2"]
    
    df["color_asymmetry_index"] = df["tbp_lv_radial_color_std_max"] * df["tbp_lv_symm_2axis"]
    new_num_cols += ["color_asymmetry_index"]
    
    df["volume_approximation_3d"] = df["tbp_lv_areaMM2"] * np.sqrt(df["tbp_lv_x"]**2 +
                                                                   df["tbp_lv_y"]**2 +
                                                                   df["tbp_lv_z"]**2)
    new_num_cols += ["volume_approximation_3d"]
    
    df["color_range"] = (np.abs(df["tbp_lv_L"] - df["tbp_lv_Lext"]) +
                         np.abs(df["tbp_lv_A"] - df["tbp_lv_Aext"]) +
                         np.abs(df["tbp_lv_B"] - df["tbp_lv_Bext"]))
    new_num_cols += ["color_range"]
    
    df["shape_color_consistency"] = df["tbp_lv_eccentricity"] * df["tbp_lv_color_std_mean"]
    new_num_cols += ["shape_color_consistency"]
    
    df["border_length_ratio"] = df["tbp_lv_perimeterMM"] / np.sqrt(2 * df["tbp_lv_areaMM2"])
    new_num_cols += ["border_length_ratio"]
    
    df["age_size_symmetry_index"] = (df["age_approx"] *
                                     df["clin_size_long_diam_mm"] *
                                     df["tbp_lv_symm_2axis"])
    new_num_cols += ["age_size_symmetry_index"]
    
    df["age_area_symmetry"] = (df["age_approx"] *
                               df["tbp_lv_areaMM2"] *
                               df["tbp_lv_symm_2axis"])
    new_num_cols += ["age_area_symmetry"]
    
    for col in numerical_features:
        df, feature_name = norm_feature(df, col)
        new_num_cols += [feature_name]
    
    df["num_images"] = df.patient_id.map(df.groupby(group_column)[id_column].count())
    new_num_cols += ["num_images"]

    return df, new_num_cols

In [4]:
INPUT_PATH = Path("/kaggle/input/isic-2024-challenge/")

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False, na_values=["NA"])
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv", low_memory=False, na_values=["NA"])

folds_df = pd.read_csv("/kaggle/input/isic-scd-folds/folds.csv")
train_metadata = train_metadata.merge(folds_df, on=[id_column, group_column], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

train_metadata = preprocess(train_metadata)
test_metadata = preprocess(test_metadata)

train_metadata, new_num_cols = feature_engineering(train_metadata)
test_metadata, _ = feature_engineering(test_metadata)

# train_images = h5py.File(INPUT_PATH / "train-image.hdf5", mode="r")
# test_images = h5py.File(INPUT_PATH / "test-image.hdf5", mode="r")

Train data size: (401059, 56)
Test data size: (3, 44)


In [5]:
# def test_augment(image_size, mean=None, std=None):
#     if mean is not None and std is not None:
#         normalize = A.Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0)
#     else:
#         normalize = A.Normalize(max_pixel_value=255.0, p=1.0)
#     transform = A.Compose(
#         [A.Resize(image_size, image_size), normalize, ToTensorV2()], p=1.0
#     )
#     return transform


# class ISICDataset(Dataset):
#     def __init__(self, metadata, images, augment, infer=False):
#         self.metadata = metadata
#         self.images = images
#         self.augment = augment
#         self.length = len(self.metadata)
#         self.infer = infer

#     def __len__(self):
#         return self.length

#     def __getitem__(self, index):
#         row = self.metadata.iloc[index]
#         image = np.array(Image.open(BytesIO(self.images[row["isic_id"]][()])))
#         if self.augment is not None:
#             image = self.augment(image=image)["image"].float()
#         if self.infer:
#             return image
#         else:
#             target = torch.tensor(row["target"])
#             return image, target

    
# class ISICNet(nn.Module):
#     def __init__(
#         self,
#         model_name,
#         pretrained=True,
#     ):
#         super(ISICNet, self).__init__()
#         self.model = create_model(
#             model_name=model_name,
#             pretrained=pretrained,
#             in_chans=3,
#             num_classes=0,
#             global_pool="",
#         )
#         in_dim = self.model.num_features
#         self.classifier = nn.Linear(in_dim, 1)
#         self.dropouts = nn.ModuleList([nn.Dropout(0.5) for _ in range(5)])

#     def forward(self, images):
#         x = self.model(images)
#         bs = len(images)
#         pool = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
#         if self.training:
#             logits = 0
#             for i in range(len(self.dropouts)):
#                 logits += self.classifier(self.dropouts[i](pool))
#             logits = logits / len(self.dropouts)
#         else:
#             logits = self.classifier(pool)
#         return logits
    

# def get_trans(img, iteration):
#     if iteration >= 6:
#         img = img.transpose(2, 3)
#     if iteration % 6 == 0:
#         return img
#     elif iteration % 6 == 1:
#         return torch.flip(img, dims=[2])
#     elif iteration % 6 == 2:
#         return torch.flip(img, dims=[3])
#     elif iteration % 6 == 3:
#         return torch.rot90(img, 1, dims=[2, 3])
#     elif iteration % 6 == 4:
#         return torch.rot90(img, 2, dims=[2, 3])
#     elif iteration % 6 == 5:
#         return torch.rot90(img, 3, dims=[2, 3])

    
# def predict(model, test_dataloader, accelerator, n_tta, log_interval=10):
#     model.eval()
#     test_probs = []
#     total_steps = len(test_dataloader)
#     with torch.no_grad():
#         for step, images in enumerate(test_dataloader):
#             logits = 0
#             probs = 0
#             for i in range(n_tta):
#                 logits_iter = model(get_trans(images, i))
#                 logits += logits_iter
#                 probs += torch.sigmoid(logits_iter)
#             logits /= n_tta
#             probs /= n_tta

#             probs = accelerator.gather(probs)
#             test_probs.append(probs)

#             if (step == 0) or ((step + 1) % log_interval == 0):
#                 print(f"Step: {step + 1}/{total_steps}")

#     test_probs = torch.cat(test_probs).cpu().numpy()
#     return test_probs

In [6]:
# class PAUC:
#     def get_final_error(self, error, weight):
#         return error

#     def is_max_optimal(self):
#         return True

#     def evaluate(self, approxes, target, weight):
#         y_true = target.astype(int)
#         y_pred = approxes[0].astype(float)
        
#         score = compute_pauc(y_true, y_pred, min_tpr=0.8)
        
#         return score, 1.0


def pauc_80(y_train, y_pred):
    score_value = compute_pauc(y_train, y_pred, min_tpr=0.8)   
    return score_value


def get_boosting_predictions(train, test, test_images, model_name, version, path):
    start_time = time.time()
    with open(path / f"{model_name}_{version}_run_metadata.json", "r") as f:
        run_metadata = json.load(f)
    pprint(run_metadata)
    
    with open(path / f"{model_name}_{version}_encoder.joblib", "rb") as f:
        mixed_encoded_preprocessor = joblib.load(f)

    enc = mixed_encoded_preprocessor.fit(train)
    X_test = enc.transform(test)

    columns_for_model = len(X_test.columns)
    print(f"Total number of columns: {columns_for_model}")
        
    all_folds = np.unique(train[fold_column])
#     all_folds = [1]
    test_predictions_df = pd.DataFrame({id_column: test[id_column]})
    for fold in all_folds:
        model_filepath = path / f"models/{model_name}_{version}_fold_{fold}.txt"
        with open(model_filepath, "rb") as f:
            estimator = joblib.load(f)
        test_predictions_df[f"fold_{fold}"] = estimator.predict_proba(X_test)[:, -1]
    test_predictions_df[target_column] = test_predictions_df[[f"fold_{fold}" for fold in all_folds if fold > 0]].mean(axis=1)
    end_time = time.time()
    return test_predictions_df[[id_column, target_column]], (end_time - start_time)


# def get_dnn_predictions(train, test, test_images, model_name, version, path):
#     start_time = time.time()
#     test_df = test[[id_column]].copy()
#     with open(path / f"{model_name}_{version}_run_metadata.json", "r") as f:
#         run_metadata = json.load(f)
#     pprint(run_metadata["params"])
    
#     image_size = run_metadata["params"]["image_size"]
#     batch_size = run_metadata["params"]["val_batch_size"]
#     fold_method = run_metadata["params"]["fold_method"]
    
#     test_dataset = ISICDataset(
#         test_df, test_images, augment=test_augment(image_size), infer=True
#     )
#     test_dataloader = DataLoader(
#         test_dataset,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=4,
#         drop_last=False,
#         pin_memory=True,
#     )
    
#     if fold_method == "gkf":
#         print("Using GroupKFold")
#         fold_column = "gkf_fold"
#     elif fold_method == "sgkf":
#         print("Using StratifiedGroupKFold")
#         fold_column = "sgkf_fold"
#     else:
#         raise ValueError(f"Fold method {fold_method} not supported")
    
#     all_folds = np.unique(train[fold_column])
# #     all_folds = [1]
#     test_predictions_df = pd.DataFrame({id_column: test[id_column]})
#     for fold in all_folds:
#         if fold < 0:
#             continue
#         print(f"\nFold {fold}")
#         accelerator = Accelerator(
#             mixed_precision=run_metadata["params"]["mixed_precision"],
#         )
        
#         model = ISICNet(model_name=model_name, pretrained=False)
#         model = model.to(accelerator.device)
        
#         model, test_dataloader = accelerator.prepare(model, test_dataloader)
#         model_filepath = path / f"models/fold_{fold}"
#         accelerator.load_state(model_filepath)

#         test_predictions_df[f"fold_{fold}"] = predict(model, test_dataloader, accelerator, n_tta=run_metadata["params"]["n_tta"])
#     test_predictions_df[target_column] = test_predictions_df[[f"fold_{fold}" for fold in all_folds if fold > 0]].mean(axis=1)
#     end_time = time.time()
#     return test_predictions_df[[id_column, target_column]], (end_time - start_time)

In [7]:
model_predict_function_topology = {
    "lgb": get_boosting_predictions,
    "cb": get_boosting_predictions,
    "xgb": get_boosting_predictions,
#     "efficientnet_b0": get_dnn_predictions, 
#     "efficientnet_b1": get_dnn_predictions,
#     "efficientnet_b2": get_dnn_predictions,
#     "efficientnet_b3": get_dnn_predictions
}

In [8]:
ensemble_preds = 0
previous_model_name = None
previous_version = None
total_runtime = 0
for idx, (model_name, version, path, weight) in enumerate(zip(model_names, versions, paths, weights)):
    print(f"Generating predictions for {model_name}_{version}")
    if test_metadata.shape[0] == 3:
        model_preds_df, runtime = model_predict_function_topology[model_name](train_metadata, 
                                                                              train_metadata.sample(n=SAMPLE_SIZE, random_state=42), 
                                                                              None, 
                                                                              model_name, 
                                                                              version, 
                                                                              Path(path))
    else:
        model_preds_df, runtime = model_predict_function_topology[model_name](train_metadata, 
                                                                              test_metadata, 
                                                                              None, 
                                                                              model_name, 
                                                                              version, 
                                                                              Path(path))
    if idx == 0:
        ensemble_preds_df = model_preds_df.copy()
    else:
        ensemble_preds_df = ensemble_preds_df.merge(model_preds_df, on=id_column, how="inner", suffixes=(f"_{previous_model_name}_{previous_version}", ""))
    ensemble_preds += ensemble_preds_df[target_column].rank(pct=True).values * weight
    previous_model_name = model_name
    previous_version = version
    total_runtime += runtime
    print("\n")
ensemble_preds_df.rename(columns={target_column: f"{target_column}_{previous_model_name}_{previous_version}"}, inplace=True)
ensemble_preds_df[target_column] = ensemble_preds

factor = EXPECTED_TEST_SIZE / SAMPLE_SIZE
expected_total_runtime = total_runtime * factor
total_runtime_minutes = int(expected_total_runtime // 60)
total_runtime_seconds = expected_total_runtime % 60
print(f"Expected total runtime during submission: {total_runtime_minutes} mins and {total_runtime_seconds} secs")

Generating predictions for xgb_v1
{'best_num_rounds': {'fold_1': 82,
                     'fold_2': 173,
                     'fold_3': 20,
                     'fold_4': 142,
                     'fold_5': 77},
 'config': {'_key': None,
            '_parent': None,
            '_temp': False,
            'model_name': 'xgb_v1',
            'models_output_dir': 'models',
            'sampling_ratio': 0.01,
            'seed': 2022},
 'cv_auc_avg': 0.9673757713901316,
 'cv_auc_oof': 0.9464921630675764,
 'cv_auc_std': 0.006521637731286478,
 'cv_pauc_avg': 0.17368981947890508,
 'cv_pauc_oof': 0.15296119492851015,
 'cv_pauc_std': 0.00587196296972231,
 'es_rounds': 150,
 'num_rounds': 2000,
 'params': {'alpha': 0.6779926606782505,
            'colsample_bylevel': 0.5476090898823716,
            'colsample_bynode': 0.9928601203635129,
            'colsample_bytree': 0.8437772277074493,
            'disable_default_eval_metric': True,
            'enable_categorical': True,
            'lambd

In [9]:
ensemble_preds_df.head()

,isic_id,target_xgb_v1,target_xgb_v2,target
0,ISIC_6973879,0.007072,0.002608,7.126374
1,ISIC_5407194,0.005565,0.001158,0.735987
2,ISIC_5273739,0.008066,0.005405,9.034119
3,ISIC_0802250,0.006882,0.002557,6.884040
4,ISIC_8084953,0.117304,0.086990,12.216288


In [10]:
ensemble_preds_df[target_column].describe()

count    5000.000000
mean        6.228822
std         3.553643
min         0.002491
25%         3.136555
50%         6.198359
75%         9.260465
max        12.452864
Name: target, dtype: float64

In [11]:
ensemble_preds_df[[id_column, target_column]].head()

,isic_id,target
0,ISIC_6973879,7.126374
1,ISIC_5407194,0.735987
2,ISIC_5273739,9.034119
3,ISIC_0802250,6.884040
4,ISIC_8084953,12.216288


In [12]:
ensemble_preds_df[[id_column, target_column]].to_csv("submission.csv", index=False)